In [28]:
import re
import string
import nltk
import spacy
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk import Tree
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score as acs
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.utils.np_utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

In [8]:
def get_data(filename):
    with open(filename ,'r') as f:
        raw_text = f.read()
    return raw_text

filename = "../Dataset/got1.txt"
data= get_data(filename)

In [13]:
data [:1000]

'A Game Of Thrones \nBook One of A Song of Ice and Fire \nBy George R. R. Martin \nPROLOGUE \n"We should start back," Gared urged as the woods began to grow dark around them. "The wildlings are \ndead." \n"Do the dead frighten you?" Ser Waymar Royce asked with just the hint of a smile. \nGared did not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. \n"Dead is dead," he said. "We have no business with the dead." \n"Are they dead?" Royce asked softly. "What proof have we?" \n"Will saw them," Gared said. "If he says they are dead, that\'s proof enough for me." \nWill had known they would drag him into the quarrel sooner or later. He wished it had been later rather \nthan sooner. "My mother told me that dead men sing no songs," he put in. \n"My wet nurse said the same thing, Will," Royce replied. "Never believe anything you hear at a woman\'s \ntit. There are things to be learned even from the dead." His voice echoed, too loud in the twilit fores

In [9]:
def lower_func(x):
    return x.strip().lower()
def remove_url(x):
 result = re.sub(r'http\S+', '', x)
 return result

def remove_hashtag(x):
 result = re.sub(r'#\S+', '', x)
 return result

def remove_mentions(x):
 result = re.sub(r'@\S+', '', x)
 return result

def remove_punc_func(x):
    new = ''
    for a in x:
        if a not in string.punctuation:
            new = new + a
        else :
            new = new +' '
    return new


## 2.2 Remove special characters
def remove_emojis(x):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    result = emoji_pattern.sub(r'', x)
    return result

def remove_spec_char_func(x):
    new=''
    for a in x:
        if (a.isalnum()==True) or (a==' '):
            new=new+a
        else :
            new = new +' '
    return new

def remove_html(x):
    c = re.compile(r'<.*?>')
    result=re.sub(c,'',x)
    return result

In [10]:
def clean_text(x):
    x= lower_func(x)
    x=remove_url(x)
    x=remove_hashtag(x)
    x=remove_mentions(x)
    x=remove_punc_func(x)
    x=remove_emojis(x)
    x=remove_spec_char_func(x)
    x=remove_html(x)
    return x

In [11]:
data_cleaned=clean_text(data)

In [12]:
data_cleaned[:1000]

'a game of thrones  book one of a song of ice and fire  by george r  r  martin  prologue   we should start back   gared urged as the woods began to grow dark around them   the wildlings are  dead     do the dead frighten you   ser waymar royce asked with just the hint of a smile   gared did not rise to the bait  he was an old man  past fifty  and he had seen the lordlings come and go    dead is dead   he said   we have no business with the dead     are they dead   royce asked softly   what proof have we     will saw them   gared said   if he says they are dead  that s proof enough for me    will had known they would drag him into the quarrel sooner or later  he wished it had been later rather  than sooner   my mother told me that dead men sing no songs   he put in    my wet nurse said the same thing  will   royce replied   never believe anything you hear at a woman s  tit  there are things to be learned even from the dead   his voice echoed  too loud in the twilit forest   page 1   we 

In [15]:
split_data=data_cleaned.split()
print(split_data[0:100])

['a', 'game', 'of', 'thrones', 'book', 'one', 'of', 'a', 'song', 'of', 'ice', 'and', 'fire', 'by', 'george', 'r', 'r', 'martin', 'prologue', 'we', 'should', 'start', 'back', 'gared', 'urged', 'as', 'the', 'woods', 'began', 'to', 'grow', 'dark', 'around', 'them', 'the', 'wildlings', 'are', 'dead', 'do', 'the', 'dead', 'frighten', 'you', 'ser', 'waymar', 'royce', 'asked', 'with', 'just', 'the', 'hint', 'of', 'a', 'smile', 'gared', 'did', 'not', 'rise', 'to', 'the', 'bait', 'he', 'was', 'an', 'old', 'man', 'past', 'fifty', 'and', 'he', 'had', 'seen', 'the', 'lordlings', 'come', 'and', 'go', 'dead', 'is', 'dead', 'he', 'said', 'we', 'have', 'no', 'business', 'with', 'the', 'dead', 'are', 'they', 'dead', 'royce', 'asked', 'softly', 'what', 'proof', 'have', 'we', 'will']


In [46]:
window_size=50
l =window_size+1
tokens =[w for w in split_data]
sequences_lines=[]
for i in range (l,len(tokens)):
    seq=tokens[i-l:i]
    line = ' '.join(seq)
    sequences_lines.append(line)

sequences_lines

['a game of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint',
 'game of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of',
 'of thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hint of a',
 'thrones book one of a song of ice and fire by george r r martin prologue we should start back gared urged as the woods began to grow dark around them the wildlings are dead do the dead frighten you ser waymar royce asked with just the hi

In [20]:
tokenizer =Tokenizer()

In [21]:
tokenizer.fit_on_texts(sequences_lines)

In [22]:
sequences=tokenizer.texts_to_sequences(sequences_lines)
vocab_size=len(tokenizer.word_index)+1

In [23]:
sequences

[[4,
  1104,
  5,
  1782,
  1309,
  49,
  5,
  4,
  1032,
  5,
  589,
  2,
  250,
  70,
  3300,
  1264,
  1264,
  2365,
  11906,
  61,
  182,
  1354,
  59,
  1031,
  1194,
  17,
  1,
  786,
  256,
  3,
  1086,
  227,
  129,
  33,
  1,
  2237,
  57,
  152,
  62,
  1,
  152,
  1990,
  10,
  41,
  1224,
  785,
  148,
  19,
  242,
  1,
  2543],
 [1104,
  5,
  1782,
  1309,
  49,
  5,
  4,
  1032,
  5,
  589,
  2,
  250,
  70,
  3300,
  1264,
  1264,
  2365,
  11906,
  61,
  182,
  1354,
  59,
  1031,
  1194,
  17,
  1,
  786,
  256,
  3,
  1086,
  227,
  129,
  33,
  1,
  2237,
  57,
  152,
  62,
  1,
  152,
  1990,
  10,
  41,
  1224,
  785,
  148,
  19,
  242,
  1,
  2543,
  5],
 [5,
  1782,
  1309,
  49,
  5,
  4,
  1032,
  5,
  589,
  2,
  250,
  70,
  3300,
  1264,
  1264,
  2365,
  11906,
  61,
  182,
  1354,
  59,
  1031,
  1194,
  17,
  1,
  786,
  256,
  3,
  1086,
  227,
  129,
  33,
  1,
  2237,
  57,
  152,
  62,
  1,
  152,
  1990,
  10,
  41,
  1224,
  785,
  148,
  19,
  242

In [24]:
sequences=np.array(sequences)

In [30]:
X,y= sequences[:,:-1],sequences[:,-1]

In [31]:
y=to_categorical(y,num_classes=vocab_size)

In [32]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [33]:
seq_length=X.shape[1]
seq_length

50

In [34]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [39]:
model = Sequential()
model.add(Embedding(vocab_size,50 , input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [40]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            595350    
                                                                 
 lstm_2 (LSTM)               (None, 50, 100)           60400     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 11907)             1202607   
                                                                 
Total params: 1,948,857
Trainable params: 1,948,857
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(X,y,batch_size=64,epochs=20)

Epoch 1/20
4670/4670 [==============================] - 352s 74ms/step - loss: 6.3723 - accuracy: 0.0709
Epoch 2/20
4670/4670 [==============================] - 386s 83ms/step - loss: 5.8517 - accuracy: 0.0991
Epoch 3/20
4670/4670 [==============================] - 312s 67ms/step - loss: 5.6059 - accuracy: 0.1138
Epoch 4/20
4670/4670 [==============================] - 317s 68ms/step - loss: 5.4375 - accuracy: 0.1216
Epoch 5/20
4670/4670 [==============================] - 330s 71ms/step - loss: 5.3019 - accuracy: 0.1281
Epoch 6/20
4670/4670 [==============================] - 332s 71ms/step - loss: 5.1958 - accuracy: 0.1340
Epoch 7/20
4670/4670 [==============================] - 331s 71ms/step - loss: 5.0998 - accuracy: 0.1384
Epoch 8/20
4670/4670 [==============================] - 333s 71ms/step - loss: 5.0180 - accuracy: 0.1428
Epoch 9/20
4670/4670 [==============================] - 329s 70ms/step - loss: 4.9433 - accuracy: 0.1469
Epoch 10/20
4670/4670 [==============================] 

In [43]:
import pickle

model.save("../Model/text_generation.h5")
pickle.dump(tokenizer,open("../Model/tokenizer.pkl","wb"))

In [65]:
from keras_preprocessing.sequence import pad_sequences


def generate_text(model,tokenizer,seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre')

        predicted = model.predict(encoded, verbose=0)
        predicted = np.argmax(predicted,axis=1)

        output_word = ''
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        in_text +=' '+output_word
        result.append(output_word)
    return ' '.join(result)

In [69]:
seed_text=sequences_lines[np.random.randint(0,len(sequences_lines))]
print(seed_text+"\n")

the darkness spun dizzily around him when he pulled his hand out golden kernels slid from between his fingers into the air they fell with him the crow landed on his hand and began to eat are you really a crow bran asked are you really falling the crow asked back



In [71]:
generate_text(model,tokenizer,seq_length,seed_text,50)

'to the realm and the king s justice the king s voice was a man s peace and i will not be a man to be a man to be a man to be a man to be a man s own time i m not a man of the'